**`Webinar 2024-03-06`**

[YouTube Recording](https://www.youtube.com/watch?v=jcS9Xac71UY&list=PLePKvgYhNOFxg3O7gXfmxaKp0EWXEhvNC&index=13)

# Augmenting Knowledge Bases with Web Search Results

### Two main approaches

##### 1. Semi-automatic augmentation using chat endpoint

In this approach, we will basically have a conversation with an AI that has web access and choose from the cited results.

After each interaction, we will send a list of the `web_search_references` we want to add to the KB.

##### 2. Full-automatic augmentation using document review + query endpoints

This is similar to a "look-alike" audience you might see when doing digital marketing.

Basically, we create our smaller KB using files we deem our "source of truth" and run document review on this KB.

The output of document review is a list of queries we will feed into the query endpoint with no KBs, but with a web search turned on.

We will add all `web_search_references` to a new KB.


### NOTE: I am separating our "source of truth" knowledge bases from the web-augmented KBs so we can always separate the manual KBs from automated KBs. When using our endpoints, we can generally pass in a list of knowledge bases so the "source of truth" and "web-augmented" KBs can still be used together.

In [1]:
# initialize variables
import superpowered
import requests
import time
import tqdm

from IPython.display import display, Markdown

SP_BASE_URL = 'https://api.superpowered.ai/v1'
SP_API_KEY_ID = ''
SP_API_KEY_SECRET = ''
API_AUTH = (SP_API_KEY_ID, SP_API_KEY_SECRET)
superpowered.set_api_key(SP_API_KEY_ID, SP_API_KEY_SECRET)

SOURCE_OF_TRUTH_KB_ID = '13ff0f75-4a20-4c7d-b68a-42799694fc65'


##### Helper functions

In [2]:
# helper functions
def run_and_display_review(kb_id: dict, ar_instructions: str, fr_instructions: str):
    url = f'{SP_BASE_URL}/knowledge_bases/{kb_id}/review'
    payload = {
        'active_reading_instructions': ar_instructions,
        'final_review_instructions': fr_instructions,
        'active_reading_model': 'claude-instant-1',
    }
    resp = requests.post(url, json=payload, auth=API_AUTH)
    final_review_display = display(Markdown('Review pending...'), display_id=True)
    progress_bar = tqdm.tqdm(total=1)
    progress_bar.set_description(f'Active Reading Progress')

    # keep request job until it's completed (or failed)
    while resp.json()['status'] not in ['COMPLETE', 'FAILED']:
        resp = requests.get(resp.json()['status_url'], auth=API_AUTH)
        progress_bar.n = resp.json()['response']['active_reading_progress_pct']
        if progress_bar.n >= 1:
            final_review_display.update(Markdown(resp.json()["response"]["final_review"]))
        progress_bar.refresh()

    progress_bar.close()
    return resp.json()['response']['final_review']


def run_chat_loop(thread_id: str, knowledge_base_id: str):
    interaction_display = display(Markdown('Conversation pending...'), display_id=True)
    added_urls = []
    while True:
        url = f'{SP_BASE_URL}/chat/threads/{thread_id}/get_response'
        user_input = input('Web search: ')
        if user_input.lower() == 'exit':
            break
        payload = {
            'input': user_input,
            'async': True,
        }
        display_header_text = f'##### `[URLS ADDED]`: {", ".join(added_urls)}\n\n----------------------------------------------------\n\n'
        display_header_text += f'##### `[USER]`: {user_input}\n\n----------------------------------------------------\n\n'
        resp = requests.post(url, json=payload, auth=API_AUTH)
        # keep request job until it's completed (or failed)
        while resp.json()['status'] not in ['COMPLETE', 'FAILED']:
            resp = requests.get(resp.json()['status_url'], auth=API_AUTH)
            display_text = display_header_text + f'##### `[SUPERPOWERED]`: {resp.json()["response"]["interaction"]["model_response"]["content"]}\n\n'
            interaction_display.update(Markdown(display_text))

        # display the web search references in markdown format
        web_search_references = resp.json()['response']['interaction']['web_search_references']
        display_text += '##### `[WEB SEARCH RESULTS]`\n\n'
        for i in web_search_references:
            title = resp.json()['response']['interaction']['web_search_results'][i]['title']
            url = resp.json()['response']['interaction']['web_search_results'][i]['url']
            display_text += f'**[{i}]** {title}: [{url}]({url})\n\n'
        interaction_display.update(Markdown(display_text))

        # choose which of these URLs to add to the knowledge base
        chosen_urls = input('Please list the numbers of the web search results you wish to add to the KB, separated by commas: ')
        chosen_urls = chosen_urls.split(',')
        chosen_urls = [resp.json()['response']['interaction']['web_search_results'][int(i)]['url'] for i in chosen_urls]
        added_urls.extend(chosen_urls)
        for url in chosen_urls:
            superpowered.create_document_via_url(knowledge_base_id, url)
        

def run_queries_through_web_search_and_add_to_kb(knowledge_base_id: str, questions: list):
    num_queries = len(questions)
    url_display = display(Markdown('**URLs added to KB**:\n\n'), display_id=True)
    display_header = '**URLs added to KB**:\n\n'
    dispaly_url_text = ''
    progress_bar = tqdm.tqdm(total=num_queries)
    progress_bar.set_description(f'Queries run:')

    all_urls = []
    for question in questions:
        url = f'{SP_BASE_URL}/knowledge_bases/query'
        payload = {
            'query': question,
            'use_web_search': True,
            'web_search_config': {
                'timeframe_days': 365 * 5,
                'exclude_domains': ['youtube.com'],
            },
            'summarize_results': True,
            'async': True,
        }
        resp = requests.post(url, json=payload, auth=API_AUTH)
        while resp.json()['status'] not in ['COMPLETE', 'FAILED']:
            resp = requests.get(resp.json()['status_url'], auth=API_AUTH)

        for i in resp.json()['response']['web_search_references']:
            url = resp.json()['response']['web_search_results'][i]['url']
            dispaly_url_text += f'[{url}]({url})\n\n'
            url_display.update(Markdown(display_header + dispaly_url_text))
            all_urls.append(url)
            try:
                superpowered.create_document_via_url(knowledge_base_id, url)
            except superpowered.exceptions.InvalidRequestError:
                pass
        progress_bar.update(1)
    progress_bar.close()

## 1. Semi-automatic augmentation using chat endpoint

In [3]:
chat_web_augmented_kb = superpowered.create_knowledge_base('Chat Web Augmented KB', 'This is a knowledge base augmented with web search results')
doc_review_query_augmented_kb = superpowered.create_knowledge_base('Document Review + Query Augmented KB', 'This is a knowledge base augmented with web search results')
thread = superpowered.create_chat_thread(
    use_web_search=True,
    web_search_exclude_domains=['youtube.com'],
    web_search_timeframe_days=365
)

In [4]:
run_chat_loop(thread['id'], chat_web_augmented_kb['id'])

##### `[URLS ADDED]`: 

----------------------------------------------------

##### `[USER]`: Can you tell me about silvopasture and agroforestry and maybe touch on their similarities and differences

----------------------------------------------------

##### `[SUPERPOWERED]`: Silvopasture and agroforestry are both sustainable land management practices that integrate trees, livestock, and crops in a mutually beneficial way. Here are some key points about each:

**Silvopasture:**
- Silvopasture is a type of agroforestry that combines trees, forage, and livestock grazing in a single system.
- It involves planting trees or managing existing trees in pastures to provide shade, shelter, and additional forage for livestock.
- Silvopasture systems can help improve soil health, reduce erosion, sequester carbon, and increase biodiversity.
- Livestock grazing in silvopasture systems can benefit from the shade provided by trees, leading to improved animal welfare and potentially increased productivity.
- Silvopasture can be a sustainable alternative to traditional grazing systems and can help farmers diversify their income streams.

**Agroforestry:**
- Agroforestry is a broader term that encompasses various land use systems that integrate trees, crops, and/or livestock.
- It includes practices such as alley cropping, windbreaks, riparian buffers, and forest farming, in addition to silvopasture.
- Agroforestry systems are designed to mimic natural ecosystems and enhance overall productivity while promoting environmental sustainability.
- Agroforestry can provide multiple benefits such as improved soil fertility, increased biodiversity, enhanced water quality, and climate change mitigation.
- Agroforestry practices are often tailored to specific environmental conditions and can be adapted to different regions and farming systems.

**Similarities:**
- Both silvopasture and agroforestry involve integrating trees with agricultural activities to achieve multiple benefits.
- They aim to improve sustainability, enhance ecosystem services, and promote resilience in agricultural systems.
- Both practices can contribute to climate change mitigation by sequestering carbon and reducing greenhouse gas emissions.
- Silvopasture is a specific type of agroforestry that focuses on combining trees, forage, and livestock, while agroforestry encompasses a wider range of integrated land use systems.

**Differences:**
- Silvopasture specifically focuses on integrating trees with livestock grazing, whereas agroforestry includes a broader range of tree-crop-livestock combinations.
- Agroforestry systems can be more diverse and complex, incorporating multiple components like crops, trees, and animals in various configurations.
- Silvopasture is more tailored towards livestock production and grazing management, while agroforestry can be applied to different types of agricultural systems beyond just grazing.

In summary, both silvopasture and agroforestry are innovative approaches to sustainable agriculture that aim to improve productivity, environmental health, and resilience in farming systems.

##### `[WEB SEARCH RESULTS]`

**[0]** Here’s to reviving the ancient practice of silvopasture | Aeon Essays: [https://aeon.co/essays/heres-to-reviving-the-ancient-practice-of-silvopasture](https://aeon.co/essays/heres-to-reviving-the-ancient-practice-of-silvopasture)

**[1]** Agroforestry: Can Farming with Trees Save the Food System?: [https://www.ediblecommunities.com/edible-stories/agroforestry-can-farming-with-trees-save-the-food-system/?utm_medium=social&utm_source=linkedin&utm_campaign=edible-stories&utm_content=civil-eats-agroforestry](https://www.ediblecommunities.com/edible-stories/agroforestry-can-farming-with-trees-save-the-food-system/?utm_medium=social&utm_source=linkedin&utm_campaign=edible-stories&utm_content=civil-eats-agroforestry)



## 2. Full-automatic augmentation using document review + query endpoints

In [5]:
active_reading_instructions = """\
Please extract any facts you deem to be important information on the best practices in the world of conservation and agriculture.
"""

final_review_instructions = """\
You are to create a list of questions from the key information you have extracted.

Each question should be on its own line with no number or anything in front of it.

You should come up with 15-20 questions.
"""

list_of_questions = run_and_display_review(SOURCE_OF_TRUTH_KB_ID, active_reading_instructions, final_review_instructions)

What are the key principles and practices that define agroecology and permaculture in the context of sustainable agriculture?
How do permaculture and agroecology incorporate traditional knowledge and practices in their approaches to farming and conservation?
In what ways do permaculture principles extend beyond those found in traditional agroecological literature?
What are the distinctive approaches to water management and crop diversity in permaculture compared to conventional agriculture?
How does the integration of terrestrial and aquatic systems in permaculture contribute to labor efficiency and multifunctionality?
What role does the configuration of agroecosystems play in enhancing the sustainability and productivity of permaculture practices?
How do permaculture principles encourage individual and community responsibility in achieving social change through sustainable agriculture?
What challenges are associated with planning and maintaining complex agroecosystems as proposed in permaculture?
How can soil disturbance from activities like logging and mining be effectively rehabilitated to restore productivity and ecological functions?
What are the benefits of using native plant species and organic amendments in soil rehabilitation efforts?
How do mechanical soil treatments like decompaction and scarification aid in the rehabilitation of disturbed soils?
What role does soil texture play in determining the effectiveness of rehabilitation techniques?
How can regenerative agriculture practices improve soil quality, increase crop yields, and enhance ecosystem services?
What are the economic and ecological benefits reported by businesses implementing regenerative agriculture programs in sub-Saharan Africa?
How do local partnerships and farmer education programs contribute to the adoption of regenerative agriculture practices?
What are the potential impacts of widespread adoption of regenerative agriculture on food security, employment, and climate mitigation in sub-Saharan Africa?
How do agroforestry practices like alley cropping and improved fallows contribute to soil fertility improvement and erosion control?
What are the challenges and considerations in adopting alley cropping systems in different climatic conditions?
How do homegardens exemplify the principles of agroforestry and contribute to food security and environmental sustainability?
What are the key factors to consider in designing and managing silvopastoral systems for improved productivity and sustainability?

Active Reading Progress: 100%|██████████| 1/1 [09:28<00:00, 568.63s/it]      


In [6]:
questions = list_of_questions.split('\n')
questions = [q for q in questions if q]  # remove empty strings

run_queries_through_web_search_and_add_to_kb(knowledge_base_id=doc_review_query_augmented_kb['id'], questions=questions)


**URLs added to KB**:

[https://www.fao.org/agroecology/overview/en](https://www.fao.org/agroecology/overview/en)

[https://www.fao.org/agroecology/knowledge/en](https://www.fao.org/agroecology/knowledge/en)

[https://www.agroecology-europe.org/our-approach/principles/](https://www.agroecology-europe.org/our-approach/principles/)

[https://www.fao.org/agroecology/overview/social-process/en](https://www.fao.org/agroecology/overview/social-process/en)

[https://library.acropolis.org/towards-permanent-co-existence-lessons-from-permaculture](https://library.acropolis.org/towards-permanent-co-existence-lessons-from-permaculture)

[https://www.mdpi.com/2073-4441/12/10/2787/pdf](https://www.mdpi.com/2073-4441/12/10/2787/pdf)

[https://www.science.org/doi/10.1126/sciadv.aba1715?cookieSet=1](https://www.science.org/doi/10.1126/sciadv.aba1715?cookieSet=1)

[https://www.frontiersin.org/articles/10.3389/fsufs.2020.00138/full](https://www.frontiersin.org/articles/10.3389/fsufs.2020.00138/full)

[https://web.archive.org/web/20200618010824/https:/iopscience.iop.org/article/10.1088/1755-1315/499/1/012018/pdf](https://web.archive.org/web/20200618010824/https:/iopscience.iop.org/article/10.1088/1755-1315/499/1/012018/pdf)

[https://nonprofitquarterly.org/fisheries-and-stewardship-lessons-from-native-hawaiian-aquaculture/](https://nonprofitquarterly.org/fisheries-and-stewardship-lessons-from-native-hawaiian-aquaculture/)

[https://www.cambridge.org/core/journals/renewable-agriculture-and-food-systems/article/abs/feasibility-and-sustainability-of-agroforestry-in-temperate-industrialized-agriculture-preliminary-insights-from-california/72C7A1E9E27CF4B22CA103AB8BF0634F](https://www.cambridge.org/core/journals/renewable-agriculture-and-food-systems/article/abs/feasibility-and-sustainability-of-agroforestry-in-temperate-industrialized-agriculture-preliminary-insights-from-california/72C7A1E9E27CF4B22CA103AB8BF0634F)

[https://www.cambridge.org/core/journals/renewable-agriculture-and-food-systems/article/abs/diversification-and-labor-productivity-on-us-permaculture-farms/7D35E31C157E70BF6B619E0471EAC81E](https://www.cambridge.org/core/journals/renewable-agriculture-and-food-systems/article/abs/diversification-and-labor-productivity-on-us-permaculture-farms/7D35E31C157E70BF6B619E0471EAC81E)

[https://sentientmedia.org/permaculture/](https://sentientmedia.org/permaculture/)

[https://permaculture.com.au/tropical-permaculture-agroforestry-central-america](https://permaculture.com.au/tropical-permaculture-agroforestry-central-america)

[https://agrilifetoday.tamu.edu/1993/10/06/secrets-of-land-reclamation-unearthed-in-lignite-mines/](https://agrilifetoday.tamu.edu/1993/10/06/secrets-of-land-reclamation-unearthed-in-lignite-mines/)

[https://www.scielo.br/j/floram/a/xxZFVnxKR3KHnMn3BdcRxCb/?lang=en](https://www.scielo.br/j/floram/a/xxZFVnxKR3KHnMn3BdcRxCb/?lang=en)

[https://www.nebiosolids.org/landreclamation](https://www.nebiosolids.org/landreclamation)

[https://www.nps.gov/articles/000/gardening-for-wildlife-with-native-plants.htm](https://www.nps.gov/articles/000/gardening-for-wildlife-with-native-plants.htm)

[https://revitalization.org/article/restoration-strategy-eight-year-experiment-proves-that-the-best-first-step-in-restoring-a-tree-species-is-not-to-plant-that-species/](https://revitalization.org/article/restoration-strategy-eight-year-experiment-proves-that-the-best-first-step-in-restoring-a-tree-species-is-not-to-plant-that-species/)

[https://openpress.usask.ca/soilscience/chapter/soil-reclamation-and-remediation-of-disturbed-lands/](https://openpress.usask.ca/soilscience/chapter/soil-reclamation-and-remediation-of-disturbed-lands/)

[https://extension.umd.edu/resource/soil-health-drainage-and-improving-soil](https://extension.umd.edu/resource/soil-health-drainage-and-improving-soil)

[https://extension.psu.edu/understanding-tree-planting-in-construction-damaged-soils](https://extension.psu.edu/understanding-tree-planting-in-construction-damaged-soils)

[https://journals.lww.com/soilsci/abstract/1999/04000/soil_resilience__a_fundamental_component_of_soil.2.aspx](https://journals.lww.com/soilsci/abstract/1999/04000/soil_resilience__a_fundamental_component_of_soil.2.aspx)

[https://www.cbf.org/issues/agriculture/regenerative-agriculture.html](https://www.cbf.org/issues/agriculture/regenerative-agriculture.html)

[https://www.conservationfinancenetwork.org/2020/03/24/the-state-of-regenerative-agriculture-growing-with-room-to-grow-more](https://www.conservationfinancenetwork.org/2020/03/24/the-state-of-regenerative-agriculture-growing-with-room-to-grow-more)

[https://www.iucn.org/news/nature-based-solutions/202110/regenerative-agriculture-works-new-research-and-african-businesses-show-how](https://www.iucn.org/news/nature-based-solutions/202110/regenerative-agriculture-works-new-research-and-african-businesses-show-how)

[https://21acres.org/blog/regenerative-ag-a-primer/](https://21acres.org/blog/regenerative-ag-a-primer/)

[https://journals.sagepub.com/doi/10.1177/0030727021998063](https://journals.sagepub.com/doi/10.1177/0030727021998063)

[https://www.sciencedirect.com/science/article/pii/S2666389921002488](https://www.sciencedirect.com/science/article/pii/S2666389921002488)

[https://www.agrifarming.in/alley-cropping-system-functions-objectives-benefits](https://www.agrifarming.in/alley-cropping-system-functions-objectives-benefits)

[https://www.devdiscourse.com/article/science-environment/2296747-agroforestry-a-win-win-for-farmers-forests-and-the-planet](https://www.devdiscourse.com/article/science-environment/2296747-agroforestry-a-win-win-for-farmers-forests-and-the-planet)

[https://www.mdpi.com/1996-1073/12/14/2737](https://www.mdpi.com/1996-1073/12/14/2737)

[https://www.mdpi.com/2504-3900/36/1/107](https://www.mdpi.com/2504-3900/36/1/107)

[https://india.mongabay.com/2021/08/keralas-homegardens-are-a-natural-solution-for-climate-change-mitigation/](https://india.mongabay.com/2021/08/keralas-homegardens-are-a-natural-solution-for-climate-change-mitigation/)

[https://www.mdpi.com/2223-7747/12/13/2473/pdf?version=1687954259](https://www.mdpi.com/2223-7747/12/13/2473/pdf?version=1687954259)

[https://article.sciencepublishinggroup.com/pdf/10.11648.j.eeb.20210603.12.pdf](https://article.sciencepublishinggroup.com/pdf/10.11648.j.eeb.20210603.12.pdf)

[https://link.springer.com/content/pdf/10.1007/s10457-020-00494-6.pdf](https://link.springer.com/content/pdf/10.1007/s10457-020-00494-6.pdf)

[https://edis.ifas.ufl.edu/publication/FR145](https://edis.ifas.ufl.edu/publication/FR145)



Queries run:: 100%|██████████| 20/20 [02:52<00:00,  8.61s/it]
